# Chapter 20: Git Workflows for CI/CD

While Chapter 19 established the mechanics of continuous integration pipelines, the effectiveness of CI depends entirely on how development teams organize their code changes. Git workflows define the branching strategies, merge policies, and collaboration patterns that determine when and how CI pipelines trigger, what they validate, and how code progresses from development to production.

The choice of workflow impacts deployment frequency, code review thoroughness, release stability, and team scalability. This chapter examines the predominant workflows—from Feature Branching to Trunk-Based Development—analyzing their implications for CI/CD pipeline design, automation opportunities, and operational constraints.

## 20.1 Feature Branch Workflow

The Feature Branch workflow represents the foundational pattern where all development occurs in isolated branches merged back to the main branch upon completion. This approach isolates work-in-progress from production code, enabling CI validation before integration.

### Core Concepts

**Mainline Stability:**
The `main` (or `master`) branch remains deployable at all times. Developers create descriptive branches for specific features or fixes:

```bash
# Create feature branch with descriptive naming convention
git checkout -b feature/user-authentication
# or
git checkout -b feature/JIRA-123-add-login-form

# Regular commits during development
git add src/auth/
git commit -m "feat(auth): implement JWT token validation

- Add middleware for token verification
- Integrate with user service API
- Add unit tests for edge cases"

# Push to remote for CI validation
git push -u origin feature/user-authentication
```

**CI Integration Pattern:**
Pipelines trigger on branch push, validating the isolated change before merge:

```yaml
# .github/workflows/feature-branch.yml
name: Feature Branch CI

on:
  push:
    branches-ignore:
      - main
      - develop

jobs:
  validate:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Run Tests
        run: npm test
        
      - name: Build Container
        run: docker build -t myapp:${{ github.ref_name }} .
        
      - name: Security Scan
        run: trivy image myapp:${{ github.ref_name }}
```

**Advantages:**
- Isolates unstable code from production branch
- Enables parallel development without interference
- Supports granular code review via Pull Requests
- Allows feature-specific CI customization (e.g., longer timeouts for complex features)

**Challenges:**
- Long-lived branches accumulate merge conflicts
- "Integration hell" when multiple features merge simultaneously
- Delayed integration defers discovery of incompatible changes
- Requires discipline to delete merged branches promptly

### Branch Naming Conventions

Standardized naming enables automation and clarity:

```bash
# Functional categories
feature/           # New functionality (e.g., feature/payment-gateway)
bugfix/            # Non-critical fixes (e.g., bugfix/validation-error)
hotfix/            # Production-critical fixes (e.g., hotfix/security-patch-2024-001)
release/           # Release preparation (e.g., release/v2.3.0)
chore/             # Maintenance tasks (e.g., chore/update-dependencies)
docs/              # Documentation updates (e.g., docs/api-reference)

# Ticket integration
feature/JIRA-456-user-profile
bugfix/GITHUB-789-memory-leak
```

**Automation Enforcement:**
```yaml
# GitLab CI - Skip unnecessary stages for docs branches
stages:
  - validate
  - test
  - deploy

validate:docs:
  rules:
    - if: $CI_COMMIT_BRANCH =~ /^docs\//
  script:
    - markdownlint docs/
    - htmlproofer site/

validate:code:
  rules:
    - if: $CI_COMMIT_BRANCH !~ /^docs\//
  script:
    - npm run lint
    - npm run test
    - docker build .
```

## 20.2 Gitflow Workflow

Gitflow extends Feature Branching with strict structural conventions for releases, hotfixes, and parallel development tracks. Introduced by Vincent Driessen in 2010, this workflow suits teams with scheduled release cycles and multiple supported production versions.

### Branch Structure

**Persistent Branches:**
- `main`: Production releases only (tags mark releases)
- `develop`: Integration branch for features (nightly builds)

**Temporary Branches:**
- `feature/*`: Spawn from `develop`, merge back to `develop`
- `release/*`: Spawn from `develop`, merge to `main` and `develop`
- `hotfix/*`: Spawn from `main`, merge to `main` and `develop`

```bash
# Initialize Gitflow (using git-flow extension or manual)
git checkout -b develop main

# Start feature
git checkout -b feature/payment-integration develop

# Finish feature
git checkout develop
git merge --no-ff feature/payment-integration
git branch -d feature/payment-integration

# Start release
git checkout -b release/v1.2.0 develop
# Update version numbers, final testing
git checkout main
git merge --no-ff release/v1.2.0
git tag -a v1.2.0 -m "Release version 1.2.0"
git checkout develop
git merge --no-ff release/v1.2.0
git branch -d release/v1.2.0
```

### CI/CD Pipeline Mapping

Gitflow requires environment-specific pipelines:

```yaml
# GitLab CI for Gitflow
stages:
  - build
  - test
  - deploy_staging
  - deploy_production

variables:
  DOCKER_IMAGE: $CI_REGISTRY_IMAGE:$CI_COMMIT_REF_SLUG

# Feature branches: Build and test only
build:feature:
  stage: build
  script:
    - docker build -t $DOCKER_IMAGE .
    - docker push $DOCKER_IMAGE
  except:
    - main
    - develop
    - /^release\/.*/
    - /^hotfix\/.*/

# Develop branch: Deploy to staging
deploy:staging:
  stage: deploy_staging
  script:
    - helm upgrade --install myapp ./chart --set image.tag=develop
  only:
    - develop

# Release branches: Pre-production validation
test:release:
  stage: test
  script:
    - npm run test:e2e
    - npm run test:integration
  only:
    - /^release\/.*/

# Main branch: Production deployment with manual gate
deploy:production:
  stage: deploy_production
  script:
    - helm upgrade --install myapp ./chart --set image.tag=$CI_COMMIT_TAG
  only:
    - main
  when: manual  # Require explicit approval
```

**Release Branch Automation:**
```bash
#!/bin/bash
# release-preparation.sh
VERSION=$1

# Create release branch
git checkout -b release/$VERSION develop

# Version bump automation
npm version $VERSION --no-git-tag-version
git add package.json package-lock.json
git commit -m "chore(release): bump version to $VERSION"

# Generate changelog
git log --pretty=format:"- %s" $(git describe --tags --abbrev=0)..HEAD > CHANGELOG.md
git add CHANGELOG.md
git commit -m "docs: update changelog for $VERSION"

# Push for CI validation
git push -u origin release/$VERSION
```

### Hotfix Process

Critical production fixes bypass the development queue:

```bash
# Critical production issue detected
git checkout main
git checkout -b hotfix/security-patch-2024-001

# Fix committed
git commit -am "fix: patch SQL injection vulnerability"

# Test in isolation
git push origin hotfix/security-patch-2024-001
# CI runs security scans and regression tests

# Deploy to production immediately
git checkout main
git merge --no-ff hotfix/security-patch-2024-001
git tag -a v1.2.1 -m "Hotfix: Security patch"
git push origin main --tags

# Ensure fix propagates to development
git checkout develop
git merge --no-ff hotfix/security-patch-2024-001
git push origin develop

git branch -d hotfix/security-patch-2024-001
```

**CI Considerations:**
- Hotfixes trigger expedited pipelines with full regression testing but accelerated queue priority
- Dual merge requirement (main + develop) ensures no regression in next release

### When to Use Gitflow

**Appropriate for:**
- Teams with scheduled release trains (monthly/quarterly)
- Software with multiple supported versions (LTS releases)
- Regulatory environments requiring release documentation and audit trails
- Desktop or mobile applications where releases are "events"

**Avoid when:**
- Practicing Continuous Deployment (multiple daily releases)
- Maintaining microservices with independent deployment cycles
- Teams smaller than 5 developers (overhead exceeds benefit)

## 20.3 Trunk-Based Development

Trunk-Based Development (TBD) represents the modern alternative to Gitflow, emphasizing short-lived branches and frequent integration to the mainline. This workflow enables true Continuous Integration by minimizing branch divergence and merge complexity.

### Core Principles

**Short-Lived Branches:**
Branches exist for hours or days, never weeks:

```bash
# Morning: Start feature
git checkout -b feature/tiny-change main
# ... 2 hours of work ...
git commit -am "feat: add email validation"
git push origin feature/tiny-change

# Create Pull Request immediately (even if incomplete)
# CI runs in < 5 minutes due to small change scope

# Afternoon: Merge after review
git checkout main
git pull origin main
git merge feature/tiny-change
git push origin main
git branch -d feature/tiny-change
```

**Branch by Abstraction:**
For large features that would traditionally require long-lived branches, use abstraction patterns:

```python
# Instead of branch: feature/new-payment-system
# Use feature flags with abstraction

class PaymentProcessor:
    def __init__(self):
        self.use_new_system = feature_flag.enabled('new-payment-system')
        self.legacy = LegacyPaymentGateway()
        self.new = ModernPaymentGateway()
    
    def process(self, amount):
        if self.use_new_system:
            return self.new.charge(amount)
        return self.legacy.charge(amount)
```

**Feature Flags:**
Toggle incomplete functionality in production:

```yaml
# config/feature-flags.yml
features:
  new-dashboard:
    enabled: false
    allowed_users: ['qa-team@company.com']
    
  dark-mode:
    enabled: true
    rollout_percentage: 10  # Gradual rollout
```

**CI Configuration for TBD:**
```yaml
# Minimal pipeline for fast feedback
stages:
  - preflight  # < 2 minutes
  - test       # < 10 minutes
  - integration

preflight:lint:
  script:
    - lint-diff --base main  # Only lint changed files
  timeout: 2m

test:unit:
  script:
    - test-only-changed --base main  # Only test affected modules
  timeout: 10m

test:full:
  script:
    - npm test  # Full suite
  only:
    - main  # Run full suite on main only
```

### Scaled Trunk-Based Development

For larger teams (>50 developers), use release branches from trunk while keeping integration frequent:

```bash
# Main remains the trunk
git checkout main
git pull origin main

# Create release branch weekly
git checkout -b release/2024-w03 main
git push origin release/2024-w03

# Cherry-pick fixes from main to release as needed
git checkout release/2024-w03
git cherry-pick abc123  # Fix from main
git push origin release/2024-w03

# Tag release when deployed
git tag -a v1.2.3 release/2024-w03
```

**CI Strategy:**
```yaml
# Build every commit to main
build:trunk:
  rules:
    - if: $CI_COMMIT_BRANCH == "main"
  script:
    - docker build -t myapp:${CI_COMMIT_SHA:0:7} .
    - docker push myapp:${CI_COMMIT_SHA:0:7}
    - helm upgrade --install myapp ./chart --set image.tag=${CI_COMMIT_SHA:0:7}

# Release branches get their own pipeline
build:release:
  rules:
    - if: $CI_COMMIT_BRANCH =~ /^release\//
  script:
    - docker build -t myapp:${CI_COMMIT_REF_SLUG} .
    - docker push myapp:${CI_COMMIT_REF_SLUG}
    # Manual promotion to production via GitOps
```

### TBD vs. Gitflow Comparison

| Aspect | Trunk-Based | Gitflow |
|--------|-------------|---------|
| **Branch Lifetime** | Hours to days | Days to weeks |
| **Integration Frequency** | Multiple times daily | At feature completion |
| **Release Cadence** | Continuous/On-demand | Scheduled |
| **Feature Flags** | Required for WIP | Optional |
| **Merge Conflict Complexity** | Low | High |
| **CI Pipeline Speed** | Fast (incremental) | Full (comprehensive) |
| **Rollback Strategy** | Feature flag toggle | Revert merge commit |

## 20.4 Pull Request Workflows

Pull Requests (PRs) or Merge Requests (MRs) serve as the primary integration gate in modern CI/CD, combining automated validation with human code review. The PR workflow transforms version control into a collaborative quality assurance process.

### PR Lifecycle Integration

**Automated Validation Gates:**
```yaml
# GitHub Actions PR workflow
name: Pull Request Validation

on:
  pull_request:
    types: [opened, synchronize, reopened]
    paths-ignore:
      - '**/*.md'

jobs:
  validate:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
        with:
          fetch-depth: 0  # Full history for SonarQube
      
      - name: Lint Code
        run: npm run lint
      
      - name: Unit Tests
        run: npm run test:unit -- --coverage
      
      - name: Build Container
        run: docker build -t myapp:pr-${{ github.event.number }} .
      
      - name: Integration Tests
        run: |
          docker-compose -f docker-compose.test.yml up -d
          npm run test:integration
          docker-compose -f docker-compose.test.yml down
      
      - name: SonarQube Scan
        uses: SonarSource/sonarcloud-github-action@master
        env:
          GITHUB_TOKEN: ${{ secrets.GITHUB_TOKEN }}
          SONAR_TOKEN: ${{ secrets.SONAR_TOKEN }}

  security-scan:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Trivy Vulnerability Scan
        uses: aquasecurity/trivy-action@master
        with:
          scan-type: 'fs'
          scan-ref: '.'
          format: 'sarif'
          output: 'trivy-results.sarif'
      
      - name: Upload to GitHub Security
        uses: github/codeql-action/upload-sarif@v2
        with:
          sarif_file: 'trivy-results.sarif'
```

**Status Checks Enforcement:**
Repositories should require specific checks to pass before merge:

```bash
# GitHub CLI - Configure branch protection
gh api repos/owner/repo/branches/main/protection \
  --method PUT \
  -H "Accept: application/vnd.github+json" \
  -f required_status_checks[strict]=true \
  -f required_status_checks[contexts][]="validate" \
  -f required_status_checks[contexts][]="security-scan" \
  -f enforce_admins=false \
  -f required_pull_request_reviews[required_approving_review_count]=2 \
  -f restrictions[users][]="" \
  -f restrictions[teams][]=""
```

### Draft PRs and Work-in-Progress

Signal incomplete work to optimize CI resources:

```yaml
# Skip expensive tests for draft PRs
jobs:
  expensive-e2e:
    if: github.event.pull_request.draft == false
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - name: Run E2E Tests
        run: npm run test:e2e
```

**Conventional Commits in PRs:**
Enforce commit message standards via CI:

```bash
#!/bin/bash
# commitlint-pr.sh
npx commitlint --from=origin/main --to=HEAD --verbose
```

### PR Templates and Automation

Standardize PR descriptions for better review:

```markdown
<!-- .github/pull_request_template.md -->
## Description
Brief description of changes

## Type of Change
- [ ] Bug fix
- [ ] New feature
- [ ] Breaking change
- [ ] Documentation update

## Testing
- [ ] Unit tests added/updated
- [ ] Integration tests pass
- [ ] Manual testing performed

## Checklist
- [ ] Code follows style guidelines
- [ ] Self-review completed
- [ ] Documentation updated
- [ ] No new warnings
- [ ] Feature flags added if applicable

## Related Issues
Fixes #(issue)
```

**Auto-labeling:**
```yaml
# Label based on file paths
name: PR Labeler
on: [pull_request]

jobs:
  label:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/labeler@v4
        with:
          configuration-path: .github/labeler.yml
```

## 20.5 Branch Protection Rules

Branch protection enforces workflow compliance, preventing direct pushes to critical branches and ensuring quality gates pass before integration.

### Protection Mechanisms

**Required Status Checks:**
```bash
# GitLab - Protected branches with CI requirements
# Settings -> Repository -> Protected Branches

# Protected branch: main
# Allowed to push: No one (except Maintainers)
# Allowed to merge: Developers + Maintainers
# Require approval from: 2 approvers
# Require CI/CD pipeline to pass: Yes
# Require code coverage: >80%
```

**GitHub Branch Protection Rules:**
```yaml
# Configured via API or UI
protection:
  required_pull_request_reviews:
    required_approving_review_count: 2
    dismiss_stale_reviews: true
    require_code_owner_reviews: true
  required_status_checks:
    strict: true  # Require branch up-to-date before merge
    contexts:
      - "ci/tests"
      - "ci/lint"
      - "security/scan"
  enforce_admins: true
  required_linear_history: true  # Prevent merge commits (rebase only)
  allow_force_pushes: false
  allow_deletions: false
```

**CODEOWNERS Integration:**
```bash
# .github/CODEOWNERS
# Global fallback
* @org/tech-leads

# Frontend changes
/src/ui/ @org/frontend-team @alice-reviewer
/src/ui/critical/ @bob-security @alice-reviewer

# Infrastructure
/terraform/ @org/devops-team
/k8s/ @org/sre-team

# Database migrations
/migrations/ @org/dba-team @org/backend-leads
```

### Signed Commits and Verification

Cryptographic verification of commit authorship:

```bash
# Require signed commits on main
gh api repos/owner/repo/branches/main/protection \
  --method PUT \
  -f required_signatures=true

# Developer setup
git config --global user.signingkey YOUR_GPG_KEY_ID
git config --global commit.gpgsign true
git commit -S -m "feat: signed commit"
```

### Bypass Strategies

Emergency procedures for broken CI:

```yaml
# GitLab - Pipeline schedules with skip flags
deploy:production:
  script:
    - helm upgrade myapp ./chart
  only:
    - main
  when: manual
  allow_failure: false  # Normally required

# Emergency bypass (requires Owner permissions):
# Push with [skip ci] in commit message for hotfixes
# Or use GitLab "Merge when pipeline succeeds" override
```

**Audit Trail:**
All bypass events should trigger notifications:

```yaml
- name: Notify on Bypass
  if: github.event.pull_request.merged_by != github.event.pull_request.user.login
  uses: slackapi/slack-github-action@v1
  with:
    payload: |
      {
        "text": "ALERT: PR merged by different user (${{ github.event.pull_request.merged_by }}) than author"
      }
```

## 20.6 Code Review Process

Effective code review balances thoroughness with velocity, leveraging both automated checks and human judgment.

### Review Automation

**Size Limits:**
Enforce small, reviewable changes:

```yaml
# Danger.js configuration (automation for PR checks)
// dangerfile.js
import { danger, fail, warn } from "danger";

const bigPRThreshold = 500;
if (danger.github.pr.additions + danger.github.pr.deletions > bigPRThreshold) {
  warn("Large PR - consider splitting into smaller chunks");
}

// Ensure tests modified with code
const srcChanges = danger.git.modified_files.filter(f => f.includes("src/"));
const testChanges = danger.git.modified_files.filter(f => f.includes("test/"));
if (srcChanges.length > 0 && testChanges.length === 0) {
  fail("Source code changes require corresponding test updates");
}
```

**Auto-assignment:**
```yaml
# .github/auto_assign.yml
addReviewers: true
addAssignees: author
reviewers:
  - senior-dev-1
  - senior-dev-2
numberOfReviewers: 1
skipKeywords:
  - wip
  - draft
```

### Review Checklists

Standardize human review criteria:

```markdown
## Reviewer Checklist
- [ ] **Functionality**: Code works as described
- [ ] **Testing**: Unit tests cover new paths
- [ ] **Security**: No SQL injection, XSS, or credential exposure
- [ ] **Performance**: No N+1 queries or memory leaks
- [ ] **Observability**: Logging and metrics added
- [ ] **Documentation**: README/API docs updated
- [ ] **Compatibility**: Backward compatible or explicitly versioned
```

### Review Response SLA

Define expectations for review turnaround:

```yaml
# GitHub Action - Stale PR reminder
name: PR Review Reminder
on:
  schedule:
    - cron: '0 9 * * *'  # Daily at 9 AM

jobs:
  remind:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/stale@v8
        with:
          stale-pr-message: 'This PR has been awaiting review for 24 hours'
          stale-pr-label: 'needs-attention'
          days-before-stale: 1
          days-before-close: -1  # Never auto-close
          only-labels: 'awaiting-review'
```

## 20.7 Merge Strategies

The method of integrating branches impacts history clarity, bisectability, and rollback procedures.

### Merge Strategies Comparison

**1. Merge Commit (No Fast-Forward):**
```bash
git merge --no-ff feature-branch
```
- Preserves complete history and branch context
- Creates explicit merge commits
- Easier to revert entire features
- History can become complex ("git graph" resembles railroad tracks)

**2. Squash and Merge:**
```bash
git merge --squash feature-branch
git commit -m "feat: complete feature description"
```
- Consolidates feature into single commit
- Clean, linear history on main
- Loses granular commit history (useful for "wip" or "fix typo" commits)
- Makes bisecting harder if feature introduced bugs

**3. Rebase and Merge:**
```bash
git checkout feature-branch
git rebase main
git checkout main
git merge feature-branch  # Fast-forward
```
- Linear history without merge commits
- Preserves individual commit granularity
- Requires force-push to feature branch (coordinate with team)
- Rewrites history (avoid on shared branches)

### CI Implications

**Merge Commits and CI:**
When using merge commits, CI should validate the merge result, not just the branch:

```yaml
# GitHub Actions - Test the merge commit
on:
  pull_request:
    types: [opened, synchronize]

jobs:
  test:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
        with:
          ref: ${{ github.event.pull_request.merge_commit_sha }}
          # Tests the actual result of merging, not just the branch head
```

**Squash Commit Messages:**
Enforce conventional commits in squash message:

```bash
# GitHub squash merge message template
<type>(<scope>): <subject>

<body>

<footer>

# Example:
feat(auth): implement OAuth2 login

- Add Google and GitHub providers
- Implement token refresh logic
- Add session management

Closes #456
```

### Choosing Merge Strategy by Context

| Context | Recommended Strategy | Rationale |
|---------|---------------------|-----------|
| **Feature branches** | Squash and Merge | Keeps main clean, one feature = one commit |
| **Release branches** | Merge Commit | Preserve exact integration point history |
| **Hotfixes** | Rebase and Merge | Linear critical path, easy cherry-pick |
| **Long-lived projects** | Merge Commit | Full audit trail for compliance |

## 20.8 Choosing the Right Workflow

Selecting a workflow requires balancing team size, release cadence, regulatory requirements, and technical maturity.

### Decision Matrix

**Team Size Considerations:**

| Team Size | Recommended Workflow | Branch Lifetime |
|-----------|---------------------|-----------------|
| 1-3 | Trunk-Based (direct push) | Hours |
| 4-10 | Trunk-Based with PRs | 1-2 days |
| 10-50 | Feature Branch or Scaled TBD | 2-3 days |
| 50+ | Gitflow or Scaled TBD with release branches | 1 week (release), 1-2 days (features) |

**Release Cadence:**

| Cadence | Workflow | CI Focus |
|---------|----------|----------|
| Continuous (multiple daily) | Trunk-Based | Fast feedback, feature flags |
| Weekly | Trunk-Based or Feature Branch | Automated staging deploys |
| Monthly | Feature Branch or Gitflow | Release branch stabilization |
| Quarterly | Gitflow | Extensive QA gates |

### Migration Strategies

**Moving from Gitflow to Trunk-Based:**
1. **Phase 1**: Shorten release branches (2 weeks → 1 week → days)
2. **Phase 2**: Implement feature flags for all new features
3. **Phase 3**: Reduce feature branch lifetime (enforce max 3 days)
4. **Phase 4**: Automate testing to enable per-commit validation
5. **Phase 5**: Remove release branches, deploy from main

**Tooling Requirements:**
- Feature flag management system (LaunchDarkly, Unleash, or custom)
- Improved test automation (shift from manual QA to automated)
- Monitoring and observability (detect issues in production quickly)

### Hybrid Approaches

Many organizations use different workflows for different components:

```yaml
# Monorepo with mixed workflows
services/
  api-gateway/          # Trunk-Based (deployed hourly)
    .gitworkflow: trunk
    
  billing-service/      # Gitflow (monthly releases)
    .gitworkflow: gitflow
    
  legacy-monolith/      # Feature Branch (quarterly)
    .gitworkflow: feature-branch
```

**CI Adaptation:**
```yaml
# Detect workflow type and adjust pipeline
stages:
  - determine-workflow
  - build
  - deploy

determine:
  script:
    - WORKFLOW=$(cat .gitworkflow 2>/dev/null || echo "feature-branch")
    - echo "WORKFLOW=$WORKFLOW" >> build.env
  artifacts:
    reports:
      dotenv: build.env

build:trunk:
  rules:
    - if: $WORKFLOW == "trunk"
  script:
    - fast-build.sh

build:gitflow:
  rules:
    - if: $WORKFLOW == "gitflow"
  script:
    - comprehensive-build.sh
```

---

## Chapter Summary and Preview

In this chapter, we explored how Git workflows structure the submission and integration of code changes into version control systems. We examined the Feature Branch workflow as the foundational pattern isolating development work, and Gitflow as its structured extension supporting scheduled releases through dedicated release and hotfix branches. We analyzed Trunk-Based Development as the modern approach enabling continuous deployment through short-lived branches, feature flags, and frequent integration. Pull Request workflows combine automated CI validation with human code review, utilizing status checks, draft PRs, and templates to standardize quality gates. Branch protection rules enforce these workflows mechanically, preventing direct pushes and requiring signed commits or specific approvals. The code review process balances automation (size limits, assignment) with human judgment via structured checklists. Finally, merge strategies—merge commit, squash, and rebase—offer trade-offs between history preservation and linear clarity, with selection criteria based on team size, release frequency, and compliance requirements.

**Key Takeaways:**
- Trunk-Based Development enables the shortest feedback loops and supports Continuous Deployment, but requires robust feature flag systems and comprehensive automated testing to safely integrate incomplete features
- Gitflow suits regulated environments and teams with infrequent release trains, but introduces merge complexity and delays integration feedback
- Branch protection with required status checks is non-negotiable for production code; bypass mechanisms must trigger audit alerts
- Squash merging maintains a clean mainline history suitable for automated changelog generation, while merge commits preserve complete context for complex features requiring rollback
- Code review automation should enforce size limits and test coverage, but human review remains essential for architectural and security validation

**Next Chapter Preview:**
Chapter 21: CI with Docker bridges version control workflows to container technologies. We will explore how Docker builds integrate into CI pipelines, examining multi-stage builds in automated environments, layer caching strategies for ephemeral CI runners, and Docker-in-Docker security considerations. This chapter details how to construct pipelines that build, scan, and push container images to the registries established in Chapter 11, completing the journey from code commit to packaged artifact ready for Kubernetes deployment. We will address platform-specific concerns for building multi-architecture images and optimizing build times through registry-based cache utilization.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='19. ci_fundamentals.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='21. ci_with_docker.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
